In [1]:
import argparse
import os

from tensorflow.keras.datasets import mnist

import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch

In [2]:
def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    import tensorflow as tf

    batch_size = 128
    num_classes = 10
    epochs = 12

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train, x_test = x_train / 255.0, x_test / 255.0
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(lr=config["lr"], momentum=config["momentum"]),
        metrics=["accuracy"],
    )

    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        validation_data=(x_test, y_test),
        callbacks=[TuneReportCallback({"mean_accuracy": "accuracy"})],
    )

In [3]:
def tune_mnist(num_training_iterations):
    
    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()

    analysis = tune.run(
        train_mnist,
        name="exp",
        metric="mean_accuracy",
        mode="max",
        stop={"mean_accuracy": 0.99, "training_iteration": num_training_iterations},
        search_alg=algo,
        num_samples=50,
        scheduler=scheduler,
        resources_per_trial={"cpu": 2, "gpu": 0},
        config={
            "lr": tune.uniform(0.001, 0.1),
            "momentum": tune.uniform(0.1, 0.9),
#             "hidden": tune.randint(32, 512), # TODO: BO does not support integer
        },
    )
    print("Best hyperparameters found were: ", analysis.best_config)


In [4]:
ray.init(num_cpus=8)
tune_mnist(num_training_iterations=10)

2022-02-25 18:45:12,991	WARNING function_runner.py:562 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-02-25 18:45:12,999	WARNING bayesopt.py:393 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2022-02-25 18:45:13,000	WARNING bayesopt.py:393 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,lr,momentum
train_mnist_fa53b8c6,PENDING,,0.0380795,0.860571


Trial name,status,loc,lr,momentum
train_mnist_fa53b8c6,RUNNING,10.99.252.58:23295,0.0380795,0.860571
train_mnist_fa700f08,RUNNING,10.99.252.58:23296,0.0734674,0.578927
train_mnist_fa834f3c,RUNNING,10.99.252.58:23297,0.0164458,0.224796
train_mnist_fa964efc,RUNNING,10.99.252.58:23302,0.00675028,0.792941


(train_mnist pid=23295) 2022-02-25 18:45:20.340173: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(train_mnist pid=23295) 2022-02-25 18:45:20.340269: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: c2178
(train_mnist pid=23295) 2022-02-25 18:45:20.340303: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: c2178
(train_mnist pid=23295) 2022-02-25 18:45:20.340461: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.82.1
(train_mnist pid=23295) 2022-02-25 18:45:20.340519: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.82.1
(train_mnist pid=23295) 2022-02-25 18:45:20.340534: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.82.1
(train_mnist pid=23295) 2022-02-25 18:45:20.341091: I tensorflow/core

Result for train_mnist_fa53b8c6:
  date: 2022-02-25_18-45-22
  done: false
  experiment_id: 85a9cf2f1dbe4909a2fd6dee5d9c929a
  hostname: c2178
  iterations_since_restore: 1
  mean_accuracy: 0.8849166631698608
  node_ip: 10.99.252.58
  pid: 23295
  time_since_restore: 5.576559543609619
  time_this_iter_s: 5.576559543609619
  time_total_s: 5.576559543609619
  timestamp: 1645832722
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: fa53b8c6
  
Result for train_mnist_fa834f3c:
  date: 2022-02-25_18-45-22
  done: true
  experiment_id: d60817bacaec4914be3a90b56c0a8d08
  hostname: c2178
  iterations_since_restore: 1
  mean_accuracy: 0.7450666427612305
  node_ip: 10.99.252.58
  pid: 23297
  time_since_restore: 5.081793308258057
  time_this_iter_s: 5.081793308258057
  time_total_s: 5.081793308258057
  timestamp: 1645832722
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: fa834f3c
  
Result for train_mnist_fa700f08:
  date: 2022-02-25_18-45-22
  done: false
  experi

...

Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_6df56bb2,RUNNING,10.99.252.58:29047,0.0999384,0.882419,0.975017,5,8.33718
train_mnist_0100171e,TERMINATED,10.99.252.58:23301,0.00303786,0.875928,0.755583,1,5.06537
train_mnist_010cc9dc,TERMINATED,10.99.252.58:23299,0.0834118,0.269871,0.858983,1,5.00529
train_mnist_04803ec8,TERMINATED,10.99.252.58:23298,0.0190007,0.246724,0.76695,1,4.96033
train_mnist_067142b8,TERMINATED,10.99.252.58:24047,0.03112,0.519805,0.8315,1,4.97889
train_mnist_08280dda,TERMINATED,10.99.252.58:24078,0.0437626,0.332983,0.831917,1,3.9203
train_mnist_0cc76d22,TERMINATED,10.99.252.58:24269,0.0676235,0.524646,0.866517,1,4.98684
train_mnist_0d67d8de,TERMINATED,10.99.252.58:24268,0.0384327,0.859025,0.96285,4,8.38249
train_mnist_0f38d8fc,TERMINATED,10.99.252.58:24331,0.0110345,0.517334,0.75325,1,4.00818
train_mnist_0fe65360,TERMINATED,10.99.252.58:24330,0.0505753,0.855785,0.978733,10,17.6021


Result for train_mnist_6df56bb2:
  date: 2022-02-25_18-48-44
  done: false
  experiment_id: 49aab233400c40cf9ebeaa3cacc6e81f
  hostname: c2178
  iterations_since_restore: 8
  mean_accuracy: 0.9818999767303467
  node_ip: 10.99.252.58
  pid: 29047
  time_since_restore: 10.775568723678589
  time_this_iter_s: 0.8128712177276611
  time_total_s: 10.775568723678589
  timestamp: 1645832924
  timesteps_since_restore: 0
  training_iteration: 8
  trial_id: 6df56bb2
  
Result for train_mnist_6df56bb2:
  date: 2022-02-25_18-48-46
  done: true
  experiment_id: 49aab233400c40cf9ebeaa3cacc6e81f
  hostname: c2178
  iterations_since_restore: 10
  mean_accuracy: 0.9829833507537842
  node_ip: 10.99.252.58
  pid: 29047
  time_since_restore: 12.418346166610718
  time_this_iter_s: 0.8206753730773926
  time_total_s: 12.418346166610718
  timestamp: 1645832926
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 6df56bb2
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_fa53b8c6,TERMINATED,10.99.252.58:23295,0.0380795,0.860571,0.9781,10,13.2933
train_mnist_fa700f08,TERMINATED,10.99.252.58:23296,0.0734674,0.578927,0.95545,4,8.02896
train_mnist_fa834f3c,TERMINATED,10.99.252.58:23297,0.0164458,0.224796,0.745067,1,5.08179
train_mnist_fa964efc,TERMINATED,10.99.252.58:23302,0.00675028,0.792941,0.77995,1,4.89102
train_mnist_ff7328be,TERMINATED,10.99.252.58:23300,0.0605104,0.666458,0.8769,1,4.98904
train_mnist_0100171e,TERMINATED,10.99.252.58:23301,0.00303786,0.875928,0.755583,1,5.06537
train_mnist_010cc9dc,TERMINATED,10.99.252.58:23299,0.0834118,0.269871,0.858983,1,5.00529
train_mnist_04803ec8,TERMINATED,10.99.252.58:23298,0.0190007,0.246724,0.76695,1,4.96033
train_mnist_067142b8,TERMINATED,10.99.252.58:24047,0.03112,0.519805,0.8315,1,4.97889
train_mnist_08280dda,TERMINATED,10.99.252.58:24078,0.0437626,0.332983,0.831917,1,3.9203


2022-02-25 18:48:46,470	INFO tune.py:636 -- Total run time: 213.48 seconds (212.28 seconds for the tuning loop).


Best hyperparameters found were:  {'lr': 0.09993841882798077, 'momentum': 0.8824186364337766}
